En el presente trabajo se analizaron 3 algoritmos distintos con el objetivo de calcular los pesos de la matriz W. <br>

$WX = y$ <br>

$ W $ = matriz de pesos <br>
$ X $ = matriz de embeddings <br>
$ Y $ = matriz de resultados <br>

Se despejó la matriz $W$ calculando la Pseudo-Inversa de la matriz de embeddings $(X^{+})$, pero utilizando diferentes descomposiciones matriciales.

$W = YX^{+}$ <br>

#### Importamos librerías

In [2]:
import importlib
import alc
importlib.reload(alc)

from alc import multiplicacionMatricialConNumpy, traspuestaConNumpy

#### Lectura de Datos y separación de casos Train y Validation

In [ ]:
from alc import cargarDataset

Xt, Yt, Xv, Yv = cargarDataset("ENTREGA/")


---

A continuación, antes de presentar el punto 6, se calcula W mediante los 3 algortimos elaborados.

#### **Algoritmo 1**

In [ ]:
from alc import calculaCholesky

A = multiplicacionMatricialConNumpy(Xt, traspuestaConNumpy(Xt))
L = calculaCholesky(A)

In [ ]:
from alc import pinVEcuacionesNormales

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
W_cholesky = pinVEcuacionesNormales(Xt, L, Yt)

#### **Algoritmo 2**

In [ ]:
#Cálculo de la SVD de X con módulo:
from alc import svd_reducida

U, S, Vt = svd_reducida(Xt)

In [55]:
from alc import pinvSVD

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
W_SVD = pinvSVD(U, S, Vt, Yt)

#### **Algoritmo 3**

**QR**

In [ ]:

from alc import calculaQR

XtTraspuesta = traspuestaConNumpy(Xt)

QHH, RHH = calculaQR (XtTraspuesta, metodo = "RH", retornanops = False)

In [62]:
from alc import pinvHouseHolder

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
W_HH = pinvHouseHolder(QHH, RHH, Yt)

**GS**

In [ ]:
from alc import calculaQR

XtTraspuesta = traspuestaConNumpy(Xt)

QGS, RGS = calculaQR (XtTraspuesta, metodo = "GS", retornanops = False)


In [67]:
from alc import pinvGramSchmidt

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
WGS = pinvGramSchmidt(QGS, RGS, Yt)

---

### 6. Evaluación y Benchmarking
a. Para cada método de resolución de la W de los items anteriores, <br>
generar una matriz de confusión evaluando a partir de los pares de embeddings de validación o testing (Xv, Yv).

In [6]:
def accuracy(tp,tn,fp,fn):
    acertados = tp + tn
    total = tp + tn + fp + fn

    return ((acertados/total)*100)

In [7]:
def matriz_confusion(Y_pred):
    tp = 0  # Predice gato y era gato
    fp = 0  
    tn = 0  # Predice perro y era perro
    fn = 0  

    n = Y_pred.shape[1]
    for i in range(n):
        # aproximamois al que más se acerque ya que nunca va a dar exactamente valores pert a {0,1}
        # 0 = gato, 1 = perro
        if Y_pred[0, i] >= Y_pred[1, i]:
            pred = 0
        else:
            pred = 1

        if Yv[0, i] == 1:
            real = 0
        else:
            real = 1

        if pred == 0 and real == 0:
            tp += 1
        elif pred == 0 and real == 1:
            fp += 1
        elif pred == 1 and real == 1:
            tn += 1
        elif pred == 1 and real == 0:
            fn += 1

    return tp, tn, fp, fn


#### Algoritmo 1 - Cholesky

In [8]:
#Calculamos predicciones de Y
Y_pred_Cholesky = W_cholesky @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_Cholesky)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 1 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracyCholesky = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 1 =",accuracyCholesky,"%")

Matriz de confusión Algoritmo 1 =
[[334, 166]]
[[150, 350]]
Accuracy Algoritmo 1 = 68.4 %


Matriz de confusión Algoritmo 1 = <br>
[[334, 166]]<br>
[[150, 350]]<br>
Accuracy Algoritmo 1 = 68.4 %

#### Algoritmo 2 - SVD

In [56]:
#Calculamos predicciones de Y
Y_pred_SVD = W_SVD @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_SVD)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 2 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracySVD = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 2 =",accuracySVD,"%")

Matriz de confusión Algoritmo 2 =
[[333, 167]]
[[152, 348]]
Accuracy Algoritmo 2 = 68.10000000000001 %


Matriz de confusión Algoritmo 2 = <br>
[[333, 167]] <br>
[[152, 348]] <br>
Accuracy Algoritmo 2 = 68.1 %

#### Algoritmo 3 - Descomposición QR

In [ ]:
#Calculamos predicciones de Y
Y_pred_HH = W_HH @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_HH)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 3 HH =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracyHH = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 3 HH =",accuracyHH,"%")

Matriz de confusión Algoritmo 3 HH = <br>
[[334, 166]] <br>
[[150, 350]] <br>
Accuracy Algoritmo 3 HH = 68.4 %

In [69]:
#Calculamos predicciones de Y
Y_pred_GS = WGS @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_GS)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 3 GS =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracyGS = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 3 GS=",accuracyGS,"%")

Matriz de confusión Algoritmo 3 GS =
[[334, 166]]
[[150, 350]]
Accuracy Algoritmo 3 GS= 68.4 %


Matriz de confusión Algoritmo 3 GS = <br>
[[334, 166]] <br>
[[150, 350]] <br>
Accuracy Algoritmo 3 GS= 68.4 %

---

b. Presentar una tabla comparativa de los resultados de cada metodología donde en las columnas se debe mostrar la performance en clasificación.

| | Cholesky | SVD | QR - HouseHolder | QR - Gram-Schmidt
|------|------|------|------|------|
| tiempo para  W (m)  | 23 | 60 | 12 | 12 |
| tiempo total (m)  | 118 | 101 | 55 | 27 |
| accuracy (%) | 68.4 | 68. 1 | 68.4 | 68.4 |

#### **Algortimo 1**
El algoritmo 1 corrió en un tiempo de total de $\simeq$ 118 minutos.
- 94 minutos para calcular Xt @ Xt.T = A y calculaCholesky(A).
- 23 minutos para calcular W con pinVEcuacionesNormales().

Observaciones: <br>
- Dentro de la función calculaCholesky(A) se utilizó la implementación del grupo de la multiplicación matricial "multiplicacionMatricialConNumpy" y se cree que es por esto que tardó 94 minutos.
- Se utilizó la traspuesta y multiplicación matricial de Numpy "@" y ".T" para obtener estos tiempos, tanto como para calcular "A" como dentro del algoritmo 1 "pinVEcuacionesNormales". <br>



#### **Algortimo 2**
El algoritmo 2 corrió en un tiempo de total de $\simeq$ 101 minutos.
- 41 minutos para calcular la SVD reducida de Xt.
- 60 minutos para calcular W con pinvSVD().

Observaciones: <br>
- Para calcular la SVD reducida de Xt se utilizó la implementación del grupo de la multiplicación matricial "multiplicacionMatricialConNumpy" y "TraspuestaConNumpy", se cree que esto elevó su tiempo (41 minutos)
- Se utilizó la multiplicación matricial implementada "multiplicacionMatricialConNumpy" dentro del algoritmo 2 "pinvSVD()". Para probar, utilizamos "@" de Numpy y se vió que el algoritmo corría en 2.1 segundos.


#### **Algortimo 3**

- HouseHolder corrió en un tiempo de total de $\simeq$ 55 minutos.
    - 43 minutos para calcular la descomposición QR con HH de Xt
    - 12 minutos para calcular W con pinvHouseHolder

- Gram-Schmidt corrió en un tiempo de total de $\simeq$ 27 minutos.
    - 15 minutos para calcular la descomposición QR con GS de Xt
    - 12 minutos para calcular W con pinvGramSchmidt

Observaciones: <br>
- Para calcular la descomposición QR con HH y con GS se utilizó la función "@" de Numpy.
- Dentro de la función "pinvHouseHolder" y "pinvGramSchmidt" se utilizaron las funciones "@" y ".T" de Numpy.  

---

### 7. Síntesis Final

En el presente trabajo práctico se implementaron y evaluaron cuatro algoritmos distintos para el cálculo de la matriz de pesos W en una red neuronal lineal aplicada a clasificación de imágenes de perros y gatos mediante transfer learning. Los algoritmos implementados fueron los siguientes: Ecuaciones Normales con Cholesky, Descomposición en Valores Singulares (SVD), y Descomposición QR mediante Householder y Gram-Schmidt.

Los resultados experimentales revelaron que todas las metodologías alcanzaron niveles de accuracy notablemente similares en el conjunto de validación, lo cual era esperable dado que todas convergen teóricamente a la misma solución de mínimos cuadrados. Esta consistencia en los resultados de clasificación valida la correcta implementación de los algoritmos y confirma que el problema está bien condicionado, permitiendo que diferentes aproximaciones numéricas lleguen a soluciones prácticamente equivalentes.

En cuanto a los tiempos de ejecución, notamos diferencias importantes entre los métodos, pero hay que tomar estos resultados con cuidado. El problema es que no pudimos hacer una comparación completamente justa: en algunos algoritmos tuvimos que usar funciones optimizadas de NumPy (como @ para multiplicar matrices y .T para trasponer) porque no sabíamos cuánto tiempo podrían llegar a demorar con nuestras implementaciones. En cambio, en otros métodos sí usamos solo nuestras funciones del TP. Esto hace que no podamos comparar los tiempos de forma directa porque básicamente estamos midiendo cosas distintas.

Notamos que los algoritmos 1 y 2 difieren en tiempo considerablemente del algoritmo 3 y esto se debe justamente a que comenzamos probando los algoritmos con funciones implementadas por el grupo, pero que por falta de tiempo terminamos utilizando las funciones de Numpy anteriormente mencionadas. Es por este motivo que para establecer conclusiones definitivas sobre la eficiencia computacional de cada metodología, sería necesario realizar una comparación homogénea utilizando únicamente implementaciones propias o únicamente funciones optimizadas de NumPy en todos los algoritmos. No obstante, el trabajo demuestra exitosamente que distintas descomposiciones pueden resolver eficazmente el problema de transfer learning, alcanzando resultados de clasificación equivalentes.